# Tensorflow Exercises: Linear Regression

## Scope: 
Use Tensorflow 2.0 and pandas to build a simple linear regression model and then test the results on a hold-out set.


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Simple Optimization

To "warm-up", I'll use the tensorflow framework to solve a simple optimization problem. This will be done with the analytic gradient, and the autodifferentiation procedure which is standard in Tensorflow.

First, define the function we will be using which is the Beale function:

$$ f(x,y) = (1.5-x+xy)^2 + (2.25-x+xy^2)^2 + (2.625-x+xy^3)^2$$ 

The gradient is:

$$  \nabla f(x,y) =  \begin{bmatrix}
2 x (y^6 + y^4 - 2 y^3 - y^2 - 2 y + 3) + 5.25 y^3 + 4.5 y^2 + 3 y - 12.75 \\
 6 x (x (y^5 + 0.666667 y^3 - y^2 - 0.333333 y - 0.333333) + 2.625 y^2 + 1.5 y + 0.5)
\end{bmatrix}  $$




In [27]:
def f_beale(x):
    return (tf.convert_to_tensor( (1.5-x[0]+x[0]*x[1])**2 + (2.25-x[0]+x[0]*x[1]**2)**2 + (2.625-x[0]+x[0]*x[1]**3)**2) )

def grad_beal(x):
    dx = 2*x[0]*(x[1]**6 + x[1]**4 - 2*x[1]**3 - x[1]**2-2*x[1] + 3) + 5.25*x[1]**3+4.5*x[1]**2+3*x[1]-12.75
    dy = 6*x[0]*(x[0]*(x[1]**5 + 2.0/3.0 * x[1]**3-x[1]**2-1.0/3.0 * x[1]-1.0/3.0) + 2.625*x[1]**2+1.5*x[1]+.5)
    return(tf.convert_to_tensor([dx,dy]))

def AD_beale(x):
    x = tf.Variable(x,dtype = tf.float32) 
    with tf.GradientTape(persistent = True) as dv:
        temp_beale = f_beale(x)
    dx = dv.gradient(temp_beale, x)
    return(dx)

We can see below that the results end up being the same for the derivatives:

In [29]:
grad_beal([1,1]) == AD_beale([1,1])

<tf.Tensor: shape=(2,), dtype=bool, numpy=array([ True,  True])>

From here, we will attemp to solve the problem using stochastic gradient descent:

In [53]:
opt = tf.keras.optimizers.SGD(learning_rate = .01)
#opt = tf.keras.optimizers.Adam(learning_rate = .01)
x_0 = tf.Variable([1.0,1.0],dtype = tf.float32)
func_for_opt = lambda: f_beale(x_0)

for epoch in range(1500):
    opt.minimize(func_for_opt, [x_0]).numpy()

x_0

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([2.9961617 , 0.49904037], dtype=float32)>

In [8]:
x_vals = tf.Variable([1.0,2.0,3.0],name = "x_vals")
x_vals_two = tf.Variable([2.0,3.0,4.0],name = "x_vals_two")
with tf.GradientTape() as tape:
    y = x_vals[0]**3+x_vals[2]*x_vals[1]**2 + 2*x_vals[2]+100
    z_one = x_vals[1]*(4*(y) + x_vals[0]**2)    
    y_two = x_vals_two[0]**3+x_vals_two[2]*x_vals_two[1]**2 + 2*x_vals_two[2]+100
    z_two = x_vals_two[1]*(4*(y_two) + x_vals_two[0]**2)
    z = z_one + z_two
    
a, b = tape.gradient(z , [z,x_vals_two]) 
print(b)    

tf.Tensor([156. 900. 132.], shape=(3,), dtype=float32)


## Constrained Optimization

We next solve a constrained optimization problem which is a constrained version of the Rosenbrock banana function:

$$ \min_{x,y} f(x,y)  =  (1-x)^2 + 100(y-x^2)$$

Subject to:
$$ (x-1)^3 - y + 1 <=0 $$

and:

$$ x+y-2<=0 $$


In [143]:
def f_rosenbrock(vals):
    return(tf.convert_to_tensor(  (1-vals[0])**2 + 100*(vals[1]-vals[0]**2)  ) )

def const1_rosenbrock(vals):    
    return(tf.convert_to_tensor( 1.0*((vals[0]-vals[1])**3 - vals[1] + 1)   ))

def const2_rosenbrock(vals):    
    return(tf.convert_to_tensor( 1.0*(vals[0]+vals[1]-2)   ))

def rosenbrock_lagrange(vals):
    return(  f_rosenbrock(vals)+const1_rosenbrock(vals)+const2_rosenbrock(vals)   )



In [76]:
f_rosenbrock([2.0,1.0])

<tf.Tensor: shape=(), dtype=float32, numpy=-299.0>

In [156]:
opt = tf.keras.optimizers.SGD(learning_rate = 1e-7,momentum = .9)
#opt = tf.keras.optimizers.Adam(learning_rate = 5e-6)
x_0 = tf.Variable([.1,.1 ],dtype = tf.float32)
func_for_opt = lambda: tf.abs(f_rosenbrock(x_0))


#for epoch in range(1000):
i=1
while (func_for_opt().numpy()>1e-4)&(i < 60000):
#    print(i)
    if i%1000 == 0:
        print("At iteration " + str(i) +  " current results are " + str(x_0.numpy() )+ " with function value of "+str( func_for_opt().numpy())  + ".")
    opt.minimize(func_for_opt, [x_0]).numpy()
    i+=1


In [157]:
opt = tf.keras.optimizers.SGD(learning_rate = 1e-8,momentum = .9)
#opt = tf.keras.optimizers.Adam(learning_rate = 1e-6)
x_0 = tf.Variable([1.1,1.1  ],dtype = tf.float32)
func_for_opt = lambda: tf.abs(rosenbrock_lagrange(x_0))


#for epoch in range(1000):
i=1
while (func_for_opt().numpy()>1e-6)&(i < 100000):
#    print(i)
    if i%5000 == 0:
        print("At iteration " + str(i) +  " current results are " + str(x_0.numpy() )+ " with function value of "+str( func_for_opt().numpy())  + ".")
    opt.minimize(func_for_opt, [x_0]).numpy()
    i+=1

print("Final results at iteration " + str(i) +  " where the final results are " + str(x_0.numpy() )+ " with function value of "+str( func_for_opt().numpy())  + ".") 

At iteration 5000 current results are [1.0583223 1.119438 ] with function value of 0.0006868541.
At iteration 10000 current results are [1.0583223 1.1194379] with function value of 0.000674814.
At iteration 15000 current results are [1.0583256 1.1194365] with function value of 0.00016772747.
At iteration 20000 current results are [1.058328  1.1194353] with function value of 0.00078471005.
At iteration 25000 current results are [1.0583247 1.1194369] with function value of 6.9633126e-05.
At iteration 30000 current results are [1.0583297 1.1194346] with function value of 0.0012120008.
At iteration 35000 current results are [1.0583258 1.1194364] with function value of 0.00020335615.
At iteration 40000 current results are [1.0583266 1.119436 ] with function value of 0.0004169941.
At iteration 45000 current results are [1.058322  1.1194382] with function value of 0.0007818341.
At iteration 50000 current results are [1.0583224 1.1194379] with function value of 0.00065122545.
At iteration 5500

## Load Data/Format Data

The results above show the basic steps of working through a mathematical optimization problem. At this point, we pivot to the use of the mathematical optimization framework to solve a statistical problem. 

Simple linear regression uses the mean squared error function to find a linear relationship between a set of features and an outcome. This is defined as:

$$ MSE(\beta) = \frac{\sum_{i=1}^N (y_i - \bar{\beta}X_i )^2 }{N}$$

Here, we step through a simple example of linear regression using tensorflow and an implementation of the MSE function.

The functions are defined in the code below:

In [292]:
def MSE(y_val ,x_val,weights):
    output = tf.Variable(tf.tensordot(X, weights, axes=1 ),name = "output"   )
    mse_val =  tf.reduce_mean( tf.square( output - y_val ) )
    return(mse_val)

For this numerical example, we'll use the cars data-set and fit a few different variables to predict the gas mileage. 

First step: Load the data into the environment and form the design matrix:

In [293]:
data = pd.read_csv( 'cars.csv' )
continuous_features = data[ [ "Identification.Year","Engine Information.Engine Statistics.Horsepower","Engine Information.Engine Statistics.Torque"] ].values / 100 
X = np.concatenate( [ continuous_features ] , axis=1 )
X = np.append(np.ones((X.shape[0],1) ) , X, axis=1)
Y = data[ [ 'Fuel Information.City mpg' ] ].values

# Perform basic subset selection in the code:
train_features , test_features ,train_labels, test_labels = train_test_split( X , Y , test_size=0.2 )
# Training data.
X = tf.Variable( train_features , dtype=tf.float32 )
Y = tf.Variable( train_labels , dtype=tf.float32 )                                                         
# Testing data
test_X = tf.Variable( test_features , dtype=tf.float32 ) 
test_Y = tf.Variable( test_labels , dtype=tf.float32 ) 
num_features = X.shape[1]
# Define the coefficientst that we'll be starting with:
weights = tf.Variable(tf.random.normal((num_features,1)))

To start, we begin with:

In [298]:
tf.trainable_variables() 

AttributeError: module 'tensorflow' has no attribute 'trainable_variables'

In [297]:
mse_opt = tf.keras.optimizers.SGD(learning_rate = 1e-7,momentum = .9)
#output = tf.Variable(   tf.tensordot( x_val , weights , axes=1 )   , dtype = tf.float32)
#opt = tf.keras.optimizers.Adam(learning_rate = 5e-6)
weight_vals = weights
def temp_mse(weight_vals):
    return(MSE(Y,X,weight_vals))
func_for_opt = lambda: tf.abs(temp_mse(weight_vals))
mse_opt.minimize(func_for_opt, [weight_vals]).numpy()

ValueError: No gradients provided for any variable: ['Variable:0'].

In [272]:
mse_opt = tf.keras.optimizers.SGD(learning_rate = 1e-7,momentum = .9)
output = tf.Variable(   tf.tensordot( x_val , weights , axes=1 )   , dtype = tf.float32)







#opt = tf.keras.optimizers.Adam(learning_rate = 5e-6)
weight_vals = weights


func_for_opt = lambda: tf.abs(temp_mse(output))
mse_opt.minimize(func_for_opt, [weight_vals]).numpy()
#for epoch in range(1000):
i=1
while (func_for_opt().numpy()>1e-4)&(i < 60000):
    print(i)
    mse_opt.minimize(func_for_opt, [weight_vals]).numpy()
    if i%1000 == 0:
        print("At iteration " + str(i) +  " current results are " + str(weight_vals.numpy() )+ " with function value of "+str( func_for_opt().numpy())  + ".")
    i+=1


1


ValueError: No gradients provided for any variable: ['Variable:0'].

In [193]:
func_for_opt()

<tf.Tensor: shape=(), dtype=float32, numpy=33.8419>

In [2]:
Y = tf.constant([2.0,2.0,2.0,2.0],dtype = 'float32')
y_pred = tf.Variable([1.0,2.0,2.0,2.0],dtype = 'float32')    
with tf.GradientTape() as tape:
    mse = tf.reduce_mean( tf.square( y_pred - Y ) )
grads = tape.gradient(mse, y_pred) 

NameError: name 'tf' is not defined

In [3]:
x_one = tf.constant([[1., 2., 3.]])
x_two = tf.constant([[2., 2., 3.]])

with tf.GradientTape() as tape:
    end_res = mean_squared_error(Y = x_one, y_pred = x_two)
    
hold = tape.gradient(end_res, x_one,x_two)

NameError: name 'tf' is not defined

In [4]:
#y_pred = tf.Variable([1.0,2.0,3.0,4.0],dtype = tf.float32)
#Y = tf.Variable( [1.0,2.0,3.0,4.0], dtype=tf.float32 ) 
output = tf.Variable( output, dtype=tf.float32 ) 
y_batch = tf.Variable( y_batch, dtype=tf.float32 ) 
with tf.GradientTape() as tape:
    mse_val = tf.reduce_mean( tf.square( output - y_batch ) )
    
tape.gradient(mse_val,output)

NameError: name 'tf' is not defined

In [21]:
mean_squared_error_deriv( y_batch , output)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.86452055]], dtype=float32)>

# Simple Linear Regression

The example below shows how to perform a simple linear regression in tensorflow using the cars dataset. In this case, we will use the analytic derivative of the function but later on, we will utilize auto differentiation function to step down.

In [ ]:

data = pd.read_csv( 'cars.csv' )

The features below are the variables we will use for the regression. This begins as an nd array but note that we add them to the craph as a constant.

In [62]:
continuous_features = data[ [ "Identification.Year","Engine Information.Engine Statistics.Horsepower","Engine Information.Engine Statistics.Torque"] ].values / 100 
X = np.concatenate( [ continuous_features ] , axis=1 )
X = np.append(np.ones((X.shape[0],1) ) , X, axis=1)
Y = data[ [ 'Fuel Information.City mpg' ] ].values
train_features , test_features ,train_labels, test_labels = train_test_split( X , Y , test_size=0.2 )
X = tf.constant( train_features , dtype=tf.float32 )
Y = tf.constant( train_labels , dtype=tf.float32 )                                                         
test_X = tf.constant( test_features , dtype=tf.float32 ) 
test_Y = tf.constant( test_labels , dtype=tf.float32 ) 

# Define the weights for the function:
weights = tf.random.normal((num_features+1,1))

In [43]:
X

<tf.Tensor: shape=(4060, 3), dtype=float32, numpy=
array([[20.1 ,  2.61,  2.81],
       [20.12,  1.7 ,  1.77],
       [20.1 ,  3.1 ,  3.65],
       ...,
       [20.12,  4.03,  4.17],
       [20.12,  1.85,  1.9 ],
       [20.11,  2.2 ,  2.58]], dtype=float32)>

Next, we define the functions that are being used to optimize linear regression. In this case, we use MSE:

$$ MSE(\beta) = \frac{\sum_{i=1}^N (y_i - \bar{\beta}X_i )^2 }{N}$$


The gradient of the MSE is composed of two different parts. First, the derivative with respect to the constant:

$$ \frac{\partial}{\partial\alpha }MSE(\beta)= -(y_i - \beta X_i) $$

The partial derivative for each coefficient is similarly derived:
$$ \frac{\partial}{\partial\beta_j }MSE(\beta)= -2x_j (y_i - \beta X_i) $$

The functions are defined in the code below:

In [ ]:
def mean_squared_error( Y , y_pred ):
    return tf.reduce_mean( tf.square( y_pred - Y ) )

In [68]:
def MSE(y_val ,x_val, weights):
    output = tf.Variable(   tf.tensordot( x_val , weights , axes=1 )   , dtype = tf.float32)
    mse_val = tf.reshape( tf.reduce_mean( tf.square( output - y_val ) )  , [ 1 , 1 ] )  
    return(mse_val)

In [69]:
MSE(y_val = Y, x_val = X,weights =  weights)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[593.0495]], dtype=float32)>

In [94]:
def MSE(y_val ,x_val, weights):
    output = tf.Variable(   np.tensordot( x_val , weights , axes=1 )   , dtype = np.float32)
    mse_val = tf.Variable(np.reshape( np.mean( np.square( output - y_val ) )  , [ 1 , 1 ] )   , dtype = np.float32   )
    return(mse_val)

MSE(y_val = Y, x_val = X,weights =  weights)

<tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[593.0494]], dtype=float32)>

In [96]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(weights)
    mse_result = MSE(y_val = Y, x_val = X,weights =  weights)

print(tape.gradient(mse_result, weights))

None


In [98]:
tf.Variable(   np.tensordot( X , weights , axes=1 )   , dtype = np.float32)

<tf.Variable 'Variable:0' shape=(4060, 1) dtype=float32, numpy=
array([[-7.4632063],
       [-7.8094263],
       [-6.0223713],
       ...,
       [-6.297119 ],
       [-6.919585 ],
       [-7.606163 ]], dtype=float32)>

In [101]:
def mse(true, predicted):
        return tf.reduce_mean(tf.square(true-predicted))      
    
with tf.GradientTape(persistent=True) as g:
    loss = mse(Y,  tf.Variable(   np.tensordot( X , weights , axes=1 )   , dtype = np.float32) )
    
g.gradient(loss, weights)    


continuous_features = data[ [ "Identification.Year","Engine Information.Engine Statistics.Horsepower","Engine Information.Engine Statistics.Torque"] ].values / 100 
type(continuous_features)

In [70]:
tf.Variable(   tf.tensordot( X , weights , axes=1 )   , dtype = tf.float32)

<tf.Variable 'Variable:0' shape=(4060, 1) dtype=float32, numpy=
array([[-7.4632063],
       [-7.8094263],
       [-6.022372 ],
       ...,
       [-6.297119 ],
       [-6.919585 ],
       [-7.606163 ]], dtype=float32)>

In [ ]:
def dMSE(y_val ,x_val, weights):
    

In [33]:



# Analytic gradient of the mean squared error:
def mean_squared_error_deriv( Y , y_pred ):
    return tf.reshape( tf.reduce_mean( 2 * ( y_pred - Y ) ) , [ 1 , 1 ] )    

# Generate approximation of the derivative using backpropogration:
 
# Apply matrix multiplication operation to the vector of betas. Add bias term instead of creating an additional row of the design matrix.
def h ( X , weights , bias ):
    return tf.tensordot( X , weights , axes=1 ) + bias

# Arbitrary choices. Note to self: How to optimize these for performance/convergence?
num_epochs = 10
num_samples = X.shape[0]
batch_size = 50
learning_rate = 0.001

# The data.Dataset call below makes the data available within Tensorflow and allows for transformations to be applied to it.
dataset = tf.data.Dataset.from_tensor_slices(( X , Y )) 
dataset = dataset.shuffle( 500 ).repeat( num_epochs ).batch( batch_size )
iterator = dataset.__iter__()

num_features = X.shape[1]
weights = tf.random.normal( ( num_features , 1 ) )
bias = 0

epochs_plot = list()
loss_plot = list()

for i in range( num_epochs ) :
    
    epoch_loss = list()
    for b in range( int(num_samples/batch_size) ):
        x_batch , y_batch = iterator.get_next()
   
        output = h( x_batch , weights , bias ) 
        loss = epoch_loss.append( mean_squared_error( y_batch , output ).numpy() )
    
    
        output = tf.Variable( output, dtype=tf.float32 ) 
        y_batch = tf.Variable( y_batch, dtype=tf.float32 ) 
        with tf.GradientTape() as tape:
            mse_val = tf.reshape( tf.reduce_mean( tf.square( output - y_batch ) )  , [ 1 , 1 ] )    
    
        dJ_dH = tape.gradient(mse_val,output)
    
  #      dJ_dH = mean_squared_error_deriv( y_batch , output)
#        print(dJ_dH)
        
        
        dH_dW = x_batch
        dJ_dW = tf.reduce_mean( dJ_dH * dH_dW )
        dJ_dB = tf.reduce_mean( dJ_dH )
    
        weights -= ( learning_rate * dJ_dW )
        bias -= ( learning_rate * dJ_dB ) 
        
    loss = np.array( epoch_loss ).mean()
    epochs_plot.append( i + 1 )
    loss_plot.append( loss ) 
    
    print( 'Loss is {}'.format( loss ) ) 

Loss is 32.58639144897461
Loss is 14.956770896911621
Loss is 10.775121688842773
Loss is 9.761115074157715
Loss is 9.497570991516113
Loss is 9.438419342041016
Loss is 9.424309730529785
Loss is 9.335890769958496
Loss is 9.53090763092041
Loss is 9.39068603515625


In [26]:
output_1

<tf.Variable 'Variable:0' shape=(5076, 1) dtype=float32, numpy=
array([[-21.167173],
       [-20.933979],
       [-20.933979],
       ...,
       [-22.604477],
       [-21.499138],
       [-21.499138]], dtype=float32)>

In [27]:
mean_squared_error_deriv(output_1,test_Y)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[77.11303]], dtype=float32)>

We can see the calculation of the function goes in two main steps:
1. Calculate the hat matrix using the h() function.
2. Use the h() results (output) to produce y_pred.
3. Use y_pred to calculate MSE.


# Two Stage Regression

In this part of the work, we will calculate a two stage regression. This process occurs using two different steps:

1. Define an initial regression:
$$ \hat{y_1} = \beta X$$
2. Define a second set of regression variables using the residuals from step $1$, and a second set of explanatory variables, $z$:
$$ \hat{y_2} = (y - \hat{y_1})*\omega_1 + \omega_{i\neq1}z\$$

This can be convoluted since we have to fit regression #1, and then fit the second regerssion. In this instance, we will employ backpropagation to embed the residuals from the first regression into the formulation of the second problem.

To do this, we set up the following optimization problem:

$$ min(\hat{y_2}  - y)^2$$

Such that: 
$$ min(\hat{y_1}  - y)^2 = 0$$


Simplified version for comparison: Solve regression #1 first, and the #2 next.

In [6]:
with tf.GradientTape() as tape:
    ret_val = tf.add(tf.reduce_mean( tf.square( output_1 - Y ) ) ,  tf.reduce_mean( tf.square( output_2 - Y ))) 
    
a = tape.gradient(ret_val,output_1)
print(a)

NameError: name 'output_1' is not defined

In [27]:
with tf.GradientTape() as tape:
    mse_val = tf.reshape( tf.reduce_mean( tf.square( output - y_batch ) )  , [ 1 , 1 ] )   

a = tape.gradient(mse_val,output)
print(a)

None


Test of the tape_gradient function. In the cast below, we are going to create a simple function with autodifferentiation which returns the function value, and the derivative of the final results.

In [7]:
type(test_X_two)

NameError: name 'test_X_two' is not defined

In [8]:
#tf.add(tf.reduce_mean( tf.square( y_pred_1 - Y ) ) ,  tf.reduce_mean( tf.square( y_pred_2 - Y ))

with tf.GradientTape() as tape:
    output_1 = h( test_X_one , weights_one , bias_1 )
    output_2 = h( test_X_two , weights_two , bias_2 )
    res = tf.reduce_mean( tf.square( output_1,test_Y ) )
    res = tf.add(tf.reduce_mean( tf.square( output_1,test_Y ) ) ,  tf.reduce_mean( tf.square( output_2 - test_Y )))
    
w_one_grads  = tape.gradient(output_1 , weights_one)  
print(w_one_grads)

NameError: name 'test_X_one' is not defined

In [9]:
continuous_features = data[ [ "Identification.Year","Engine Information.Engine Statistics.Horsepower"] ].values / 100 
#categorical_research_features = data[ [ 'Research' ] ].values 
step1_vars = np.concatenate( [ continuous_features ] , axis=1 )
step2_vars = np.concatenate( [ data[ ["Engine Information.Engine Statistics.Torque"] ] ], axis=1 )
dep_var = data[ [ 'Fuel Information.City mpg' ] ].values
test_X_one = tf.constant( step1_vars , dtype=tf.float32 ) 
test_X_two = tf.constant( step2_vars , dtype=tf.float32 ) 
test_Y = tf.constant( dep_var , dtype=tf.float32 )
dataset = tf.data.Dataset.from_tensor_slices(( test_X_one , test_Y )) 
dataset = dataset.shuffle( 500 ).repeat( num_epochs ).batch( batch_size )
iterator = dataset.__iter__()
num_features_one = test_X_one.shape[1]
num_features_two = test_X_two.shape[1]
weights_one = tf.random.normal( ( num_features_one , 1 ) )
weights_two = tf.random.normal( ( num_features_two , 1 ) )
lambda_var = tf.random.normal( ( 1 , 1 ) )
bias_1 = tf.random.normal( ( 1 , 1 ) )
bias_2 = tf.random.normal( ( 1 , 1 ) )
epochs_plot = list()
loss_plot = list()
output_1 = tf.Variable(h( test_X_one , weights_one , bias_1 ),tf.float32)
output_2 = tf.Variable(h( test_X_one , weights_one , bias_2 ),tf.float32)

In [12]:
with tf.GradientTape() as tape:
    
    output_1 =tf.Variable( h( test_X_one , weights_one , bias_1 ), dtype=tf.float32 )
    output_2 = tf.Variable( h( test_X_two , weights_two , bias_2 ), dtype=tf.float32 )
#    res = tf.reduce_mean( tf.square( output_1,test_Y ) )
    res = tf.reshape( tf.add(tf.reduce_mean( tf.square( output_1,test_Y ) ) ,  tf.reduce_mean( tf.square( output_2 - test_Y ))) , [ 1 , 1 ] )   
    
print(tape.gradient(res , output_1)  )


tf.Tensor(
[[-0.00669864]
 [-0.00704053]
 [-0.00704053]
 ...
 [-0.00464521]
 [-0.00626577]
 [-0.00626577]], shape=(5076, 1), dtype=float32)


In [13]:
x_vals = tf.Variable([1.0,2.0,3.0],name = "x_vals")
x_vals_two = tf.Variable([2.0,3.0,4.0],name = "x_vals_two")
with tf.GradientTape() as tape:
    y = x_vals[0]**3+x_vals[2]*x_vals[1]**2 + 2*x_vals[2]+100
    z_one = x_vals[1]*(4*(y) + x_vals[0]**2)    
    y_two = x_vals_two[0]**3+x_vals_two[2]*x_vals_two[1]**2 + 2*x_vals_two[2]+100
    z_two = x_vals_two[1]*(4*(y_two) + x_vals_two[0]**2)
    z = z_one + z_two
    
a, b = tape.gradient(z , [x_vals,x_vals_two]) 
print(b)    

tf.Tensor([156. 900. 132.], shape=(3,), dtype=float32)


In [17]:
np.mean( np.square( output_1 - test_Y ) )

1938.5131

In [18]:
tf.reduce_mean( tf.square( output_1 - test_Y ) ) 

<tf.Tensor: shape=(), dtype=float32, numpy=1938.5135>

In [20]:
w1 = tf.Variable(tf.random.normal( ( num_features_one , 1 ) ),name = "w1")
w2 = tf.Variable(tf.random.normal( ( num_features_two, 1 ) ),name = "w2")
bias_1 = tf.Variable(tf.random.normal( ( 1 , 1 ) ),name = "bias_1")
bias_2 = tf.Variable(tf.random.normal( ( 1 , 1 ) ),name = "bias_2")
output_1 =tf.Variable( h( test_X_one , w1 , bias_1 ), dtype=tf.float32 )
output_2 = tf.Variable( h( test_X_two , w2 , bias_2 ), dtype=tf.float32 )

with tf.GradientTape() as tape:
#    mse_val = tf.reshape( tf.reduce_mean( tf.square( output_1 - test_Y ) )  , [ 1 , 1 ] )
#    mse_val = np.sum(output_1 - test_Y)**2 
#    mse_val = w1[0]**2 + w1[1]*2+w1[0]*w1[1] + 10 + w2[0]**3
    mse_val =  tf.Variable(np.mean( np.square( output_1 - test_Y ) ),dtype = tf.float32)
results = tape.gradient(mse_val,w1)
print(results)

None


In [151]:
output = h( test_X_one , weights_one , bias_1 ) 

output_1 =tf.Variable( h( test_X_one , weights_one , bias_1 ), dtype=tf.float32 )
output_2 = tf.Variable( h( test_X_two , weights_two , bias_2 ), dtype=tf.float32 )
#loss = epoch_loss.append( mean_squared_error( test_Y , output_1 ).numpy() )
#output = tf.Variable( output_1, dtype=tf.float32 ) 
y_batch = tf.Variable( test_Y, dtype=tf.float32 ) 
with tf.GradientTape() as tape:
    tape.watch(weights_one)
    mse_val = tf.reshape( tf.reduce_mean( tf.square( output_1 - test_Y ) )  , [ 1 , 1 ] )    
    
dJ_dH = tape.gradient(mse_val,weights_one)
print(dJ_dH)

None


In [154]:
output_1

<tf.Variable 'Variable:0' shape=(5076, 1) dtype=float32, numpy=
array([[17.685701],
       [18.122816],
       [18.122816],
       ...,
       [15.076742],
       [17.14866 ],
       [17.14866 ]], dtype=float32)>

In [152]:
tape.watched_variables()

(<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
 array([[ 1.0401548],
        [-0.8742276]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(5076, 1) dtype=float32, numpy=
 array([[17.685701],
        [18.122816],
        [18.122816],
        ...,
        [15.076742],
        [17.14866 ],
        [17.14866 ]], dtype=float32)>)

In [ ]:
with tf.GradientTape() as tape:    
    output_1 = tf.Variable(h( test_X_one , weights_one , bias_1 ),dtype = tf.float32)
    output_2 = tf.Variable( h( test_X_two , weights_two , bias_2 ) ,dtype = tf.float32)
    
    #    final = tf.tensordot( test_X_one , weights_one , axes=1 ) + bias_1
#    output_2 = h( test_X_two , weights_two , bias_2 )
   # res = tf.reduce_mean( tf.square( output_1,test_Y ) )
#    res = tf.add(tf.reduce_mean( tf.square( output_1,test_Y ) ) ,  tf.reduce_mean( tf.square( output_2 - test_Y )))
    
w_one_grads  = tape.gradient(output_1 , weights_one)  
print(w_one_grads)

In [ ]:
with tf.GradientTape() as tape:
    ret_val = reg_lagrange( Y=y_batch , y_pred_1 = output_1, y_pred_2 = output_2 )

In [16]:
epochs_plot = list()
loss_plot = list()


# Define lagrangian here by taking the sum of the two functions.
def reg_lagrange( Y , y_pred_1, y_pred_2 ):
    return tf.add(tf.reduce_mean( tf.square( y_pred_1 - Y ) ) ,  tf.reduce_mean( tf.square( y_pred_2 - Y ))) 


for i in range( num_epochs ) :    
    epoch_loss = list()
    for b in range( int(num_samples/batch_size) ):
        x_batch , y_batch = iterator.get_next()
#output = h( x_batch , weights , bias ) 
        output_1 = h( test_X_one , weights_one , bias_1 )
        output_2 = h( test_X_one , weights_one , bias_2 )
        loss = epoch_loss.append( reg_lagrange( Y=y_batch , y_pred_1 = output_1, y_pred_2 = output_2 ).numpy() )
    
        output = tf.Variable( output, dtype=tf.float32 ) 
        y_batch = tf.Variable( y_batch, dtype=tf.float32 ) 
        with tf.GradientTape() as tape:
            mse_val = tf.reshape( tf.reduce_mean( tf.square( output - y_batch ) )  , [ 1 , 1 ] )   
    output_1
    
        with tf.GradientTape() as tape:
            ret_val = reg_lagrange( Y=y_batch , y_pred_1 = output_1, y_pred_2 = output_2 )
    
        dJ_dH = tape.gradient(mse_val,output)
        #dJ_dH = mean_squared_error_deriv( y_batch , output)
        dH_dW = x_batch
        dJ_dW = tf.reduce_mean( dJ_dH * dH_dW )
        dJ_dB = tf.reduce_mean( dJ_dH )
    
        weights -= ( learning_rate * dJ_dW )
        bias -= ( learning_rate * dJ_dB ) 
    loss = np.array( epoch_loss ).mean()
    epochs_plot.append( i + 1 )
    loss_plot.append( loss ) 

In [ ]:
continuous_features = data[ [ "Identification.Year","Engine Information.Engine Statistics.Horsepower"] ].values / 100 
#categorical_research_features = data[ [ 'Research' ] ].values 
step1_vars = np.concatenate( [ continuous_features ] , axis=1 )
step2_vars = np.concatenate( [ data[ ["Engine Information.Engine Statistics.Torque"] ] ], axis=1 )
dep_var = data[ [ 'Fuel Information.City mpg' ] ].values
test_X_one = tf.constant( step1_vars , dtype=tf.float32 ) 

num_epochs = 10
num_samples = test_X_one.shape[0]
batch_size = 50
learning_rate = 0.001


test_X_two = tf.constant( step2_vars , dtype=tf.float32 ) 
test_Y = tf.constant( dep_var , dtype=tf.float32 )

dataset = tf.data.Dataset.from_tensor_slices(( test_X_one , test_Y )) 
dataset = dataset.shuffle( 500 ).repeat( num_epochs ).batch( batch_size )
iterator = dataset.__iter__()
num_features = test_X_one.shape[1]
weights = tf.random.normal( ( num_features , 1 ) )
bias = 0

epochs_plot = list()
loss_plot = list()

for i in range( num_epochs ) :    
    epoch_loss = list()
    for b in range( int(num_samples/batch_size) ):
        x_batch , y_batch = iterator.get_next()
   
        output = h( x_batch , weights , bias ) 
        loss = epoch_loss.append( mean_squared_error( y_batch , output ).numpy() )
    
        output = tf.Variable( output, dtype=tf.float32 ) 
        y_batch = tf.Variable( y_batch, dtype=tf.float32 ) 
        with tf.GradientTape() as tape:
            mse_val = tf.reshape( tf.reduce_mean( tf.square( output - y_batch ) )  , [ 1 , 1 ] )   
    
    
        dJ_dH = tape.gradient(mse_val,output)
        #dJ_dH = mean_squared_error_deriv( y_batch , output)
        dH_dW = x_batch
        dJ_dW = tf.reduce_mean( dJ_dH * dH_dW )
        dJ_dB = tf.reduce_mean( dJ_dH )
    
        weights -= ( learning_rate * dJ_dW )
        bias -= ( learning_rate * dJ_dB ) 
    loss = np.array( epoch_loss ).mean()
    epochs_plot.append( i + 1 )
    loss_plot.append( loss ) 
    
#    print( 'Loss is {}'.format( loss ) ) 

#Get predictions and then use them to create residuals.
preds = bias +  weights[0]*step1_vars[:,0]+  weights[1]*step1_vars[:,1]     
res_vals = preds-dep_var[:,0]        
hold = pd.DataFrame(step2_vars)
hold["res"] = np.array(res_vals).tolist()
test_X_two = tf.constant( np.concatenate( [ hold ], axis=1 ) , dtype=tf.float32 ) 
 
dataset_two = tf.data.Dataset.from_tensor_slices(( test_X_two , test_Y )) 
dataset_two = dataset_two.shuffle( 500 ).repeat( num_epochs ).batch( batch_size )
iterator = dataset_two.__iter__()
num_features = test_X_two.shape[1]
weights = tf.random.normal( ( num_features , 1 ) )
bias = 0
        
for i in range( num_epochs ) :    
    epoch_loss = list()
    for b in range( int(num_samples/batch_size) ):
        x_batch , y_batch = iterator.get_next()
   
        output = h( x_batch , weights , bias ) 
        loss = epoch_loss.append( mean_squared_error( y_batch , output ).numpy() )
    
        output = tf.Variable( output, dtype=tf.float32 ) 
        y_batch = tf.Variable( y_batch, dtype=tf.float32 ) 
        with tf.GradientTape() as tape:
            mse_val = tf.reshape( tf.reduce_mean( tf.square( output - y_batch ) )  , [ 1 , 1 ] )   
    
    
        dJ_dH = tape.gradient(mse_val,output)
        #dJ_dH = mean_squared_error_deriv( y_batch , output)
        dH_dW = x_batch
        dJ_dW = tf.reduce_mean( dJ_dH * dH_dW )
        dJ_dB = tf.reduce_mean( dJ_dH )
    
        weights -= ( learning_rate * dJ_dW )
        bias -= ( learning_rate * dJ_dB )      
        
        
        
    loss = np.array( epoch_loss ).mean()
    epochs_plot.append( i + 1 )
    loss_plot.append( loss ) 
    print( 'Loss is {}'.format( loss ) ) 

The version below solves the problem as a lagrange optimization problem:


$$ min(\hat{y_2}  - y)^2 + \lambda(\hat{y_1}  - \beta X)^2$$


In [ ]:

def reg_lagrange_problem( Y , y_one_pred, y_two_pred ):
    return tf.reduce_mean( tf.square( y_pred - Y ) ) + 

In [ ]:
preds = bias +  weights[0]*step1_vars[:,0]+  weights[1]*step1_vars[:,1] 
res_vals = preds-dep_var[:,0]
res_vals = np.array(res_vals)
#res_vals.resize([5076,0])
res_vals

In [ ]:
np.array(res_vals,step2_vars[:,0])

In [ ]:
step2_vars[:,0].

In [ ]:
type(data)

In [ ]:
np.concatenate( [ data[ ["Engine Information.Engine Statistics.Torque"] ] ], axis=1 )

In [ ]:
hold = pd.DataFrame(step2_vars)
hold["res"] = res_vals.tolist()
np.concatenate( [ hold ], axis=1 )

In [ ]:
np.concatenate((step2_vars, res_vals), 1)

In [ ]:
type(res_vals)

In [ ]:
type(step2_vars)

In [ ]:
np.hstack(res_vals,step2_vars)

In [ ]:
def run_reg(indeps,deps,num_epochs = 10,batch_size = 50, learning_rate = .001):
    num_samples = indeps.shape[0]
    indeps = tf.constant( indeps , dtype=tf.float32 ) 
    deps = tf.constant( deps , dtype=tf.float32 ) 
    
    dataset = tf.data.Dataset.from_tensor_slices(( indeps , deps )) 
    dataset = dataset.shuffle( 500 ).repeat( num_epochs ).batch( batch_size )
    iterator = dataset.__iter__()
    num_features = indeps.shape[1]
    weights = tf.random.normal( ( num_features , 1 ) )
    bias = 0
    epochs_plot = list()
    loss_plot = list()
 


In [ ]:
pred_vals = bias +  weights[0]*step1_vars[:,0]+  weights[1]*step1_vars[:,1] - dep_var
pred_vals - 

In [ ]:
dep_var

In [ ]:
bias

In [ ]:
bias